In [135]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import math
import statistics

In [64]:
# County codes
fips = pd.read_csv("fips.csv", encoding="mbcs")

# Wastewater COVID-19 concentration data
wastewater = pd.read_csv("wastewater.csv")

# Hospitalized patient number data
hospital = pd.read_csv("hospital.csv")

# County populations
population = pd.read_csv("population.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9,10,14,15,27,30,31,39,52,53,65,69,80,87,89) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [65]:
# Only select California counties from national directory
fips = fips.loc[fips["state"]=="CA"]

# Remove County from name to make it consistent with hospital and population
fips["name"] = [i.replace(" County", "") for i in list(fips["name"])]

# fips to County dictionary needed to reformat wastewater county list
county_dict = dict(zip(fips["fips"], fips["name"]))

# population dictionary needed to adjust hospitalization numbers for county population
population_dict = dict(zip(population["Entity Name"], population["Estimated Population"]))

In [66]:
# Some wastewater treatment plants serve two counties. We filter these out
wastewater = wastewater.loc[wastewater["county_names"].str.len()<7]

# Change fips ids to county strings
wastewater["county_names"] = [county_dict[int(i)] for i in list(wastewater["county_names"])]

In [67]:
print(hospital.columns)
print(fips.columns)
print(population.columns)
print(wastewater.columns)

Index(['county', 'todays_date', 'hospitalized_covid_confirmed_patients',
       'hospitalized_suspected_covid_patients', 'hospitalized_covid_patients',
       'all_hospital_beds', 'icu_covid_confirmed_patients',
       'icu_suspected_covid_patients', 'icu_available_beds'],
      dtype='object')
Index(['fips', 'name', 'state'], dtype='object')
Index(['Entity Name', 'Fiscal Year', 'Estimated Population'], dtype='object')
Index(['epaid', 'wwtp_name', 'reporting_jurisdiction', 'county_names',
       'other_jurisdiction', 'zipcode', 'capacity_mgd', 'population_served',
       'industrial_input', 'stormwater_input', 'influent_equilibrated',
       'sewage_travel_time', 'institution_type', 'wwtp_jurisdiction',
       'solids_separation', 'pasteurized', 'time_zone', 'sample_id', 'lab_id',
       'sample_collect_date', 'sample_collect_time', 'flow_rate',
       'collection_water_temp', 'sample_type', 'composite_freq',
       'sample_matrix', 'sample_location', 'sample_location_specify',
       

In [68]:
# Select three columns for analysis, and drop all rows with null values
hospital = hospital.loc[:, ['county', 'todays_date', 'hospitalized_covid_patients']].dropna()

In [69]:
hospital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61208 entries, 1285 to 62492
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   county                       61208 non-null  object 
 1   todays_date                  61208 non-null  object 
 2   hospitalized_covid_patients  61208 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.9+ MB


In [71]:
wastewater.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28155 entries, 0 to 32255
Data columns (total 91 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   epaid                      27174 non-null  object 
 1   wwtp_name                  28155 non-null  object 
 2   reporting_jurisdiction     28155 non-null  object 
 3   county_names               28155 non-null  object 
 4   other_jurisdiction         0 non-null      float64
 5   zipcode                    28155 non-null  int64  
 6   capacity_mgd               28155 non-null  float64
 7   population_served          28155 non-null  int64  
 8   industrial_input           6425 non-null   float64
 9   stormwater_input           5459 non-null   object 
 10  influent_equilibrated      5459 non-null   object 
 11  sewage_travel_time         4949 non-null   float64
 12  institution_type           28155 non-null  object 
 13  wwtp_jurisdiction          28155 non-null  obj

In [72]:
for i in range(len(wastewater.columns)):
    print(str(wastewater.columns[i])+": " + str(wastewater[str(wastewater.columns[i])].unique()))

epaid: ['CA0078981' 'CA0038768' '5D150100001' 'CA7000009' 'CA0037648'
 '5D100105001' 'CA0053813' '5A170102002' 'CA0110604' 'CA0105392'
 'CA0107409' 'CA0047953' 'CA0038067' '-1' 'CA0038628' 'CA0038539'
 'CA0109991' 'CA0037958' 'CA0037541' 'CA0037621' 'CA0038369' 'CA0049964'
 'CA0037834' 'CA0077682' 'CA0037842' 'CA0037575' 'CA0037810' 'CA0037869'
 'CA0048194' 'CA0077950' 'CA0104477' 'CA0037699' 'CA0078948' 'CA00367869'
 nan 'CA0107417' 'CA0107611' 'CA0108928' 'CA0105350' 'CA0037702'
 'CA8000409' 'CA0038776' 'CA0038598' 'CA0048127' 'CA0037851' 'CA0022764'
 'CA0023345' 'CA0079103' 'CA0079219' 'CA0079049' 'CA0081558' 'CA0084271'
 'CA0079138' 'CA0079154' 'CA0079243' 'CAS004002' '5A570108001'
 '5A570104001' '5C240107001']
wwtp_name: ['AmerValley_Quincy' 'Amer_Canyon_Napa' 'BKERSFLD_P2' 'BKERSFLD_P3'
 'CALEXICO_CTY' 'CCCSD_WWTP' 'FRSN_CLVS_RWRF' 'LACSD_Jnt' 'LakeCty_SE'
 'OCSD_P1' 'SBMWD_WWRF' 'SDPU_PtLom'
 'City of Paso Robles Wastewater Treatment Plant'
 'Sausalito-Marin City Sanitary Distri

In [73]:
wastewater["hum_frac_mic_unit"].value_counts()

copies/g dry sludge          22450
copies/L wastewater           4100
log10 copies/L wastewater      535
Name: hum_frac_mic_unit, dtype: int64

In [74]:
wastewater["pcr_target_units"].value_counts()

copies/g dry sludge          22450
copies/L wastewater           4722
log10 copies/L wastewater      857
copies/l wastewater            126
Name: pcr_target_units, dtype: int64

In [75]:
# Select columns for wastewater analysis
wastewater = wastewater.loc[:, ["county_names", "sample_collect_date", "rec_eff_spike_conc", "pcr_target_units", "pcr_target_avg_conc", "pcr_target", "hum_frac_mic_unit", "hum_frac_mic_conc"]].dropna()

In [76]:
wastewater.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26220 entries, 0 to 31539
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county_names         26220 non-null  object 
 1   sample_collect_date  26220 non-null  object 
 2   rec_eff_spike_conc   26220 non-null  float64
 3   pcr_target_units     26220 non-null  object 
 4   pcr_target_avg_conc  26220 non-null  object 
 5   pcr_target           26220 non-null  object 
 6   hum_frac_mic_unit    26220 non-null  object 
 7   hum_frac_mic_conc    26220 non-null  float64
dtypes: float64(2), object(6)
memory usage: 1.8+ MB


In [77]:
# Only select wastewater columns with a copies/g unit because we don't have a g/L unit conversion
# Eliminate units columns from out analysis because we won't need them anymore
wastewater = wastewater[(wastewater['pcr_target_units'] == 'copies/g dry sludge') & (wastewater['hum_frac_mic_unit'] == 'copies/g dry sludge')].loc[:, ["county_names", "sample_collect_date", "rec_eff_spike_conc", "pcr_target_avg_conc", "pcr_target", "hum_frac_mic_conc"]]


In [78]:
wastewater.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21734 entries, 5026 to 31539
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county_names         21734 non-null  object 
 1   sample_collect_date  21734 non-null  object 
 2   rec_eff_spike_conc   21734 non-null  float64
 3   pcr_target_avg_conc  21734 non-null  object 
 4   pcr_target           21734 non-null  object 
 5   hum_frac_mic_conc    21734 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.2+ MB


In [79]:
set(wastewater["county_names"])

{'Alameda',
 'Contra Costa',
 'Los Angeles',
 'Madera',
 'Marin',
 'Merced',
 'Monterey',
 'Napa',
 'Orange',
 'Riverside',
 'Sacramento',
 'San Benito',
 'San Bernardino',
 'San Diego',
 'San Luis Obispo',
 'San Mateo',
 'Santa Barbara',
 'Santa Clara',
 'Santa Cruz',
 'Solano',
 'Sonoma',
 'Stanislaus',
 'Yolo'}

In [80]:
# Limit hospital dataset to the counties where we have wastewater data
hospital = hospital[(hospital["county"].isin(set(wastewater["county_names"].tolist())))]

In [81]:
hospital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25139 entries, 1289 to 62490
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   county                       25139 non-null  object 
 1   todays_date                  25139 non-null  object 
 2   hospitalized_covid_patients  25139 non-null  float64
dtypes: float64(1), object(2)
memory usage: 785.6+ KB


In [82]:
# Count unique pairings of hospital/date.
len(set([(hospital.iloc[i]["county"], hospital.iloc[i]["todays_date"]) for i in range(len(hospital))]))
#This value equals the size of hospital, so the hospital data is aggregated by county.

25139

In [83]:
hospital["todays_date"]

1289     2020-04-21
1290     2020-04-21
1291     2020-04-21
1298     2020-04-21
1300     2020-04-21
            ...    
62485    2023-04-18
62486    2023-04-18
62487    2023-04-18
62488    2023-04-18
62490    2023-04-18
Name: todays_date, Length: 25139, dtype: object

In [84]:
wastewater["sample_collect_date"]

5026     1/11/2022
5027     1/11/2022
5028     1/11/2022
5029     1/13/2022
5030     1/13/2022
           ...    
31535    6/10/2022
31536    6/10/2022
31537     6/7/2022
31538     6/7/2022
31539     6/9/2022
Name: sample_collect_date, Length: 21734, dtype: object

In [86]:
# We need to convert the dates to a common format
hospital["todays_date"] = [datetime.strptime(hospital.iloc[i]["todays_date"], '%Y-%m-%d') for i in range(len(hospital))]

In [87]:
wastewater["sample_collect_date"] = [datetime.strptime(wastewater.iloc[i]["sample_collect_date"], '%m/%d/%Y') for i in range(len(wastewater))]

In [89]:
hospital = hospital.sort_values(by='todays_date', ascending=True)

In [91]:
wastewater = wastewater.sort_values(by='sample_collect_date', ascending=True)

In [92]:
hospital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25139 entries, 1289 to 62490
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   county                       25139 non-null  object        
 1   todays_date                  25139 non-null  datetime64[ns]
 2   hospitalized_covid_patients  25139 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 785.6+ KB


In [93]:
wastewater.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21734 entries, 25577 to 10479
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   county_names         21734 non-null  object        
 1   sample_collect_date  21734 non-null  datetime64[ns]
 2   rec_eff_spike_conc   21734 non-null  float64       
 3   pcr_target_avg_conc  21734 non-null  object        
 4   pcr_target           21734 non-null  object        
 5   hum_frac_mic_conc    21734 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 1.2+ MB


In [259]:
# Test the functionality of adding six days to a datetime64[ns] datatype
print(hospital.iloc[133]["todays_date"])
print(hospital.iloc[133]["todays_date"]+np.timedelta64(6, 'D'))

2020-04-26 00:00:00
2020-05-02 00:00:00


In [95]:
# Create a new column for hospital overcrowding in the wastewater dataframe
wastewater['hospital_overcrowding'] = np.nan

In [101]:
# New hospital column for hospitalized covid suspected patients per capita
hospital["hospitalized_per_capita"] = [hospital.iloc[i]["hospitalized_covid_patients"]/population_dict[hospital.iloc[i]["county"]] for i in range(len(hospital))]

In [103]:
# Iterate through each row of the wastewater dataframe
for index, row in wastewater.iterrows():
    # Find the corresponding county and date in the hospital dataframe
    county = row['county_names']
    date = row['sample_collect_date'] + pd.Timedelta(days=6)
    hospital_row = hospital[(hospital['county'] == county) & (hospital['todays_date'] >= date - pd.Timedelta(hours=13)) & (hospital['todays_date'] <= date + pd.Timedelta(hours=13))]
    
    # If a corresponding row is found, copy the hospitalized_covid_patients value into the wastewater dataframe
    if not hospital_row.empty:
        wastewater.loc[index, 'hospital_overcrowding'] = hospital_row["hospitalized_per_capita"].values[0]
    else:
        wastewater.loc[index, 'hospital_overcrowding'] = None

In [98]:
wastewater.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21734 entries, 25577 to 10479
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   county_names           21734 non-null  object        
 1   sample_collect_date    21734 non-null  datetime64[ns]
 2   rec_eff_spike_conc     21734 non-null  float64       
 3   pcr_target_avg_conc    21734 non-null  object        
 4   pcr_target             21734 non-null  object        
 5   hum_frac_mic_conc      21734 non-null  float64       
 6   hospital_overcrowding  21638 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 1.8+ MB


In [102]:
hospital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25139 entries, 1289 to 62490
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   county                       25139 non-null  object        
 1   todays_date                  25139 non-null  datetime64[ns]
 2   hospitalized_covid_patients  25139 non-null  float64       
 3   hospitalized_per_capita      25139 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 982.0+ KB


In [116]:
# drops rows that do not have a hospital overcrowding value for six days later
wastewater = wastewater.dropna()

# convert the 'pcr_target_avg_conc' column to float data type
wastewater['pcr_target_avg_conc'] = wastewater['pcr_target_avg_conc'].astype(float)

# perform one hot encoding on the 'pcr_target' column of the wastewater dataframe
wastewater = pd.concat([wastewater, pd.get_dummies(wastewater['pcr_target'])], axis=1)

In [117]:
wastewater.describe()

,pcr_target_avg_conc,hum_frac_mic_conc,hospital_overcrowding,hMPXV,hMPXV Clade II,omicron,sars-cov-2
count,2.163800e+04,2.163800e+04,21638.000000,21638.000000,21638.000000,21638.000000,21638.000000
mean,1.790351e+05,1.356503e+09,0.000093,0.067982,0.107727,0.213375,0.610916
std,3.108033e+05,3.459123e+09,0.000067,0.251721,0.310043,0.409699,0.487554
min,0.000000e+00,1.938805e+07,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.086181e+03,5.954860e+08,0.000053,0.000000,0.000000,0.000000,0.000000
50%,8.401463e+04,9.611537e+08,0.000081,0.000000,0.000000,0.000000,1.000000
75%,2.376014e+05,1.520000e+09,0.000116,0.000000,0.000000,0.000000,1.000000
max,1.141656e+07,2.440000e+11,0.000741,1.000000,1.000000,1.000000,1.000000


In [129]:
wastewater.drop("rec_eff_spike_conc", axis=1, inplace=True)
wastewater.drop("county_names", axis=1, inplace=True)
# model must be date agnostic and must only rely on predictive indicators
wastewater.drop("sample_collect_date", axis=1, inplace=True)
# made redundant by one hot encoding
wastewater.drop("pcr_target", axis=1, inplace=True)

In [205]:
# Concentrations must be log scaled because they follow a log distribution
wastewater["pcr_target_avg_conc"] = np.log(wastewater["pcr_target_avg_conc"]+10)
wastewater["hum_frac_mic_conc"] = np.log(wastewater["hum_frac_mic_conc"])

In [118]:
print("RMSE of predicting the median for all values: ")
median = statistics.median(list(wastewater["hospital_overcrowding"]))
print(np.sqrt(mean_squared_error(list(wastewater["hospital_overcrowding"]), [median for i in range(len(wastewater))])))

RMSE of predicting the median for all values: 
6.806749259251654e-05


In [252]:
# feature label split
X = wastewater.drop("hospital_overcrowding", axis=1).to_numpy()
y = wastewater["hospital_overcrowding"].to_numpy()
# scale X to a standard distribution
scaler_X = StandardScaler()
scaler_X.fit(X)
scaler_X.transform(X)
# scale y to a [0, 1] interval
y_min = np.min(y)
y_max = np.max(y)
y = (y-y_min)/(y_max-y_min)
# 77% train 23% test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=1)

In [253]:
model = MLPRegressor(hidden_layer_sizes=(500, 200, 20),
                     activation = "relu",
                     learning_rate_init=0.001,
                     random_state=1,
                     max_iter=500,
                     verbose=True,
                     solver = "adam",
                     tol=1e-6,
                     batch_size=64,
                     n_iter_no_change=20)
regr = model.fit(X_train, y_train)

Iteration 1, loss = 0.00673350
Iteration 2, loss = 0.00416029
Iteration 3, loss = 0.00421474
Iteration 4, loss = 0.00415805
Iteration 5, loss = 0.00416131
Iteration 6, loss = 0.00401864
Iteration 7, loss = 0.00420282
Iteration 8, loss = 0.00397737
Iteration 9, loss = 0.00390924
Iteration 10, loss = 0.00391063
Iteration 11, loss = 0.00397927
Iteration 12, loss = 0.00390165
Iteration 13, loss = 0.00388436
Iteration 14, loss = 0.00381300
Iteration 15, loss = 0.00388055
Iteration 16, loss = 0.00380330
Iteration 17, loss = 0.00379774
Iteration 18, loss = 0.00385399
Iteration 19, loss = 0.00384864
Iteration 20, loss = 0.00375278
Iteration 21, loss = 0.00376391
Iteration 22, loss = 0.00374677
Iteration 23, loss = 0.00372280
Iteration 24, loss = 0.00372969
Iteration 25, loss = 0.00374289
Iteration 26, loss = 0.00373712
Iteration 27, loss = 0.00372785
Iteration 28, loss = 0.00371285
Iteration 29, loss = 0.00371209
Iteration 30, loss = 0.00370845
Iteration 31, loss = 0.00371329
Iteration 32, los

In [254]:
predictions = regr.predict(X_test)

In [255]:
# Reverse transform predictions and actual values ahead of error calculation
predictions = predictions*(y_max-y_min)+y_min
y_test = y_test*(y_max-y_min)+y_min
print("RMSE: ")
print(np.sqrt(mean_squared_error(predictions, y_test)))

RMSE: 
6.364677010778086e-05


In [257]:
# Compare prediction statistics with 
stats = {'min': np.min(predictions),
         'max': np.max(predictions),
         'median': np.median(predictions),
         'q1': np.percentile(predictions, 25),
         'q3': np.percentile(predictions, 75)}
print(stats)

{'min': 1.7466591914361386e-05, 'max': 0.00013959734782930977, 'median': 8.382598719548963e-05, 'q1': 7.616939330684809e-05, 'q3': 0.00010324284681824599}
